## Session 3 - TSP (i)

### Analyzing Optimal Depot Location for Colissimo

Scenario:
You are a group of students enrolled in an operations research course. Your latest assignment involves analyzing the logistics organization of Colissimo, a parcel delivery service operating in France. Colissimo wants to improve its deliveries between its various post offices in Paris, particularly in the 10th arrondissement, and is considering opening a new depot in the area. Your task is to determine the best location for this new depot among three options provided by Colissimo.

Objective:
Your objective is to minimize the traveled distance for Colissimo's delivery routes while also considering potential reductions in CO2 emissions to align with Colissimo's ecological responsibilities.

Data:
You have been provided with data describing the locations of post offices in Paris, including their addresses and geographic coordinates. Additionally, you have a distance matrix calculated based on driving distances between these locations, as well as information about the three possible depot locations: Métro Jean Jaurès, Station de métro "Couronnes," and Gare Paris Saint-Lazare.

In [ ]:
!pip3 install ortools

In [ ]:
#Distance Matrix with depot 1 and 11 locations
# [
# [0, 850, 1000, 1300, 1300, 2300, 1400, 650, 550, 1100, 550],
# [2700, 0, 2900, 1500, 3100, 2500, 3700, 1900, 2300, 260, 2000],
# [1100, 1400, 0, 1600, 1700, 2800, 1200, 1100, 1100, 1700, 1100],
# [1200, 1500, 1600, 0, 1900, 1500, 2300, 550, 1100, 1000, 700],
# [1300, 3100, 1400, 1900, 0, 3000, 1200, 1400, 1200, 1700, 1200],
# [2300, 1800, 2900, 1200, 3100, 0, 2100, 1700, 2400, 2100, 1900],
# [2000, 2700, 2500, 2100, 1900, 2000, 0, 2200, 2200, 3000, 2200],
# [750, 950, 1100, 250, 1300, 1600, 1800, 0, 600, 1200, 160],
# [550, 2300, 1100, 1100, 1200, 2300, 1500, 700, 0, 1200, 550],
# [1200, 300, 1500, 1100, 1600, 2400, 2200, 950, 650, 0, 800],
# [1300, 1400, 1600, 1400, 1900, 2400, 2300, 800, 1100, 1600,0],
# ]



# Distance matrix with Depot 2 and 11 locations
# [0, 850, 1000, 1300, 1300, 2300, 1400, 650, 550, 1100, 550, 2500],
# [2700, 0, 2900, 1500, 3100, 2500, 3700, 1900, 2300, 260, 2000, 3200],
# [1100, 1400, 0, 1600, 1700, 2800, 1200, 1100, 1100, 1700, 1100, 2200],
# [1200, 1500, 1600, 0, 1900, 1500, 2300, 550, 1100, 1000, 700, 2200],
# [1300, 3100, 1400, 1900, 0, 3000, 1200, 1400, 1200, 1700, 1200, 2500],
# [2300, 1800, 2900, 1200, 3100, 0, 2100, 1700, 2400, 2100, 1900, 1500],
# [2000, 2700, 2500, 2100, 1900, 2000, 0, 2200, 2200, 3000, 2200, 1300],
# [750, 950, 1100, 250, 1300, 1600, 1800, 0, 600, 1200, 160, 2300],
# [550, 2300, 1100, 1100, 1200, 2300, 1500, 700, 0, 1200, 550, 2500],
# [1200, 300, 1500, 1100, 1600, 2400, 2200, 950, 650, 0, 800, 2900],
# [1300, 1400, 1600, 1400, 1900, 2400, 2300, 800, 1100, 1600,0, 3100],
# [2200, 1900, 2900, 2600, 1900, 3900, 3100, 1900, 2000, 2100, 2200, 0]



# Distance matrix with depot 3 and 11 locations:
# [0, 850, 1000, 1300, 1300, 2300, 1400, 650, 550, 1100, 550, 2700],
# [2700, 0, 2900, 1500, 3100, 2500, 3700, 1900, 2300, 260, 2000, 3900],
# [1100, 1400, 0, 1600, 1700, 2800, 1200, 1100, 1100, 1700, 1100, 3700],
# [1200, 1500, 1600, 0, 1900, 1500, 2300, 550, 1100, 1000, 700, 3300],
# [1300, 3100, 1400, 1900, 0, 3000, 1200, 1400, 1200, 1700, 1200, 2000],
# [2300, 1800, 2900, 1200, 3100, 0, 2100, 1700, 2400, 2100, 1900, 4400],
# [2000, 2700, 2500, 2100, 1900, 2000, 0, 2200, 2200, 3000, 2200, 3900],
# [750, 950, 1100, 250, 1300, 1600, 1800, 0, 600, 1200, 160, 3100],
# [ 550, 2300, 1100, 1100, 1200, 2300, 1500, 700, 0, 1200, 550, 2600],
# [1200, 300, 1500, 1100, 1600, 2400, 2200, 950, 650, 0, 800, 2500],
# [1300, 1400, 1600, 1400, 1900, 2400, 2300, 800, 1100, 1600,0, 3600],
# [2200, 1900, 2900, 2600, 1900, 3900, 3100, 1900, 2000, 2100, 2200, 0]


In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


# TSP with 11 locations

def create_data_model():
    """Stores the data for the problem."""
    # address: 0: 38 Boulevard de Strasbourg, 1: 169 Rue du Faubourg St. Denis, 2: 56 Rue Rene Boulanger, 3: 228 Rue du Faubourg St. Martin, 4: 18 Boulevard de bonne nouvelle, 5: 46 Rue de Sambre et Meuse, 6: 11 Rue Leon Jouhaux, 7: 158 Rue du Faubourg St. Martin, 8: 2 square Alban Satragne, 9: 8 Rue de Dunkerque, 10: 4 Rue du 8 Mai 1945, 11: depot 1
    data = {}
    data['distance_matrix'] = [
        [0, 850, 1000, 1300, 1300, 2300, 1400, 650, 550, 1100, 550, 2300],
        [2700, 0, 2900, 1500, 3100, 2500, 3700, 1900, 2300, 260, 2000, 2000],
        [1100, 1400, 0, 1600, 1700, 2800, 1200, 1100, 1100, 1700, 1100, 2700],
        [1200, 1500, 1600, 0, 1900, 1500, 2300, 550, 1100, 1000, 700, 1100],
        [1300, 3100, 1400, 1900, 0, 3000, 1200, 1400, 1200, 1700, 1200, 3000],
        [2300, 1800, 2900, 1200, 3100, 0, 2100, 1700, 2400, 2100, 1900, 1900],
        [2000, 2700, 2500, 2100, 1900, 2000, 0, 2200, 2200, 3000, 2200, 2800],
        [750, 950, 1100, 250, 1300, 1600, 1800, 0, 600, 1200, 160, 1600],
        [550, 2300, 1100, 1100, 1200, 2300, 1500, 700, 0, 1200, 550, 2300],
        [1200, 300, 1500, 1100, 1600, 2400, 2200, 950, 650, 0, 800, 1900],
        [1300, 1400, 1600, 1400, 1900, 2400, 2300, 800, 1100, 1600 ,0, 2400],
        [1900, 1100, 2100, 550, 2400, 1200, 2900, 1100, 1700, 1400, 1300, 0]]


    data['num_vehicles'] = 1

    # The index of the depot, the location where all vehicles start and end their routes.
    data['depot'] = 0
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} m'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]


data = create_data_model()

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

routing = pywrapcp.RoutingModel(manager)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)

if solution:
        print_solution(manager, routing, solution)



Objective: 11920 m
Route for vehicle 0:
 3 -> 7 -> 10 -> 1 -> 9 -> 8 -> 0 -> 4 -> 2 -> 6 -> 5 -> 11 -> 3

